In [ ]:
! pip install gpt_index

In [ ]:
! pip install langchain

In [1]:
from llama_index import SimpleDirectoryReader,GPTListIndex,GPTVectorStoreIndex,LLMPredictor,PromptHelper
from langchain import OpenAI
import sys
import os

C:\Users\SITHARA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["OPENAI_API_KEY"] = "#####################################3"

In [4]:
from llama_index import ServiceContext,StorageContext,load_index_from_storage

def create_index(path):
    print(path)
    max_input = 4096
    token = 256
    chunk_size = 600
    max_chunk_overlap = 20

    promptHelper = PromptHelper(max_input,token,max_chunk_overlap,chunk_size_limit = chunk_size)

    # #define LLM
    llmpredictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-ada-001", max_tokens=token))

    #load data
    docs = SimpleDirectoryReader(path).load_data()
    print(docs)

    #Create vector index 
    serviceContext = ServiceContext.from_defaults(llm_predictor = llmpredictor,prompt_helper= promptHelper)

    vectorIndex = GPTVectorStoreIndex.from_documents(documents= docs,service_context= serviceContext)
    vectorIndex.storage_context.persist(persist_dir='json_data')
    return vectorIndex
    

In [5]:
vectorIndex = create_index('data_files')

data_files
[Document(text='How to connect Iot bridge with WeatherStation/DHT11?\n"Connect power connector(red and black connector) to any of the power pin(PC1 to PC11)except PC12\nand  data connection is C01 of DHT11 will connect to the C010 of  IoT bridge"\n\nHow to connect Iot bridge with Soil moisture?\n"Connect power connector(red and black connector) to any of the power pin(PC1 to PC11)except PC12\nand  data connection is C01 of Soil moisture will connect to the C07 of the IoT bridge"\n\nHow to connect Iot bridge with LDR Proximity?\n"Connect power connector(red and black connector) to any of the power pin(PC1 to PC11)except PC12\nand  data connection is LDR1 And LDR2 of LDR will connect to the C07of the IoT bridge, LDR_LED_PIN of LDR will connect to the C06 of the IoT bridge"\n\nHow to connect Iot bridge with Motors?\n"Connect power connector(red and black connector) to any of the power pin(PC1 to PC11)except PC12\nand  data connections are MOTOR DRIVER C01 will connect to the C0

In [6]:

def answerMe(questions):
    storageContext = StorageContext.from_defaults(persist_dir='json_data')
    index = load_index_from_storage(storageContext)
    quer_engine = index.as_query_engine()
    response = quer_engine.query(questions)
    return response


In [7]:
response = answerMe('please tell me how to connect motor')

In [8]:
response.response

'\n"Connect power connector(red and black connector) to any of the power pin(PC1 to PC11)except PC12\nand  data connections are MOTOR DRIVER C01 will connect to the C05 of the IoT bridge, C02 will connect to the C06 of the IoT bridge,C03 will connect to the C04 of the IoT bridge"'